In [257]:
import os
from typing import List
import numpy as np

In [258]:
DC_CLOUD = 0
DC_EDGE = 1

DS_PUBLIC = 0
DS_PRIVATE = 1

DS_INIT = 0
DS_GEN = 1

NUM_DATACENTRES = 3
NUM_DATASETS = 5
NUM_TOTAL_DATASETS = 6
NUM_TASKS = 5

In [259]:
class datacentre(object):
    def __init__(self, dc_type: int, capacity: int, index: int):
        self.capacity = capacity
        self.dc_type = dc_type
        self.index = index
        
    def __str__(self):
        return (str(self.capacity))

class dataset(object):
    def __init__(self, size: int, gt: int, lc: datacentre, flc: datacentre,
                 ds_type: int, input_ds_type: int, index: int):
        self.size = size
        self.gt = gt #Generating task
        self.lc = lc #dc location
        self.flc = flc #dc final location
        self.ds_type = ds_type #public or private
        self.input_ds_type = input_ds_type #input or generated dataset
        self.index = index

    def __str__(self):
        return (str(self.size) + str(self.ds_type))
        
        
class task(object):
    def __init__(self, ids: List[int] , ods: List[int], index: int):
        self.ids = ids #input dataasets
        self.ods = ods #output dataset
        self.index = index
        
    def __str__(self):
        return (str(self.ids) + " " + str(self.ods))
            

In [260]:
datacentres = []
datasets = []
tasks = []
task_dependencies = []
bandwidth = []

def initialize_dc():
    dc1 = datacentre(DC_CLOUD, 10000, 1)
    dc2 = datacentre(DC_EDGE, 20, 2)
    dc3 = datacentre(DC_EDGE, 20, 3)
    
    datacentres.extend([dc1, dc2, dc3])
    
def initialize_ds():
    ds1 = dataset(3, None, -1, -1, DS_PUBLIC, DS_INIT, 1)
    ds2 = dataset(5, None, -1, -1, DS_PUBLIC, DS_INIT, 2)
    ds3 = dataset(3, None, -1, -1, DS_PUBLIC, DS_INIT, 3)
    ds4 = dataset(3, None, datacentres[1], datacentres[1], DS_PRIVATE, DS_INIT, 4)
    ds5 = dataset(5, None, datacentres[2], datacentres[2], DS_PRIVATE, DS_INIT, 5)
    ds6 = dataset(8, None, -1, -1, DS_PUBLIC, DS_GEN,6)
    
    datasets.extend([ds1, ds2, ds3, ds4, ds5, ds6])
    

def initialize_tasks():
    t1 = task([datasets[0], datasets[1]], [], 1)
    t2 = task([datasets[0], datasets[1], datasets[5]], [], 2)
    t3 = task([datasets[0], datasets[1], datasets[2], datasets[5]], [], 3)
    t4 = task([datasets[2], datasets[3], datasets[5]], [], 4)
    t5 = task([datasets[4]], [datasets[5]], 5)
    
    tasks.extend([t1, t2, t3, t4, t5])
    

def initialize_task_dep():
    global task_dependencies
    task_dependencies = [[] for i in range(NUM_TASKS)]
    task_dependencies[4].extend([1,2,3])
    
    
def initialize_bandwidth():
    global bandwidth
    bandwidth = [[0] * NUM_DATACENTRES for i in range(NUM_DATACENTRES)]
    bandwidth[0][1] = bandwidth[1][0] = 10
    bandwidth[0][2] = bandwidth[2][0] = 20
    bandwidth[1][2] = bandwidth[2][1] = 150
    
initialize_dc()
initialize_ds()
initialize_tasks()
initialize_task_dep()
initialize_bandwidth()

In [261]:
#Topologically sort tasks
#Place task in datacentre with minimal transmission time
#After placing task place output tasks in same data centre
#Calculate Ttotal

    
# A recursive function used by topologicalSort
def topologicalSortUtil(v,visited,stack):

    visited[v] = True

    for i in task_dependencies[v]:
        print(i)
        if visited[i] == False:
            topologicalSortUtil(i,visited,stack)

    stack.insert(0,v)

def topologicalSort():

    visited = [False]*NUM_TASKS
    stack =[]

    for i in range(NUM_TASKS):
        if visited[i] == False:
            topologicalSortUtil(i,visited,stack)

    print(stack)
    return stack
        
sorted_tasks = topologicalSort()

1
2
3
[4, 3, 2, 1, 0]


In [262]:
def place_tasks(particle):
    dataset_to_datacenter = [0] * NUM_TOTAL_DATASETS
    for i in range(len(particle)):
        dataset_to_datacenter[i] = particle[i]
    print(dataset_to_datacenter)
        
    task_datacentres = [0] * NUM_TASKS
    transmission_times = [0] * NUM_TASKS
    
    print(task_datacentres)
    
    for i in range(NUM_TASKS):
        transmission_time = 10000
        datacentre = None
        
        task = tasks[sorted_tasks[i]]
        print("Task - " + str(task.index))
        
        private_dataset_input = 0
        for ids in task.ids:
            if(ids.ds_type == DS_PRIVATE):
                datacentre = ids.lc
                private_dataset_input = 1
                print("Private ids " + str(ids.index) + " found in datacenter " + 
                      str(dataset_to_datacenter[ids.index-1]))
        
        if(private_dataset_input == 1):
            transmission_time_i = 0
            
            for ids in task.ids:
                print("Input dataset " + str(ids.index))
                idc = dataset_to_datacenter[ids.index-1]
                print("Input dataset resides in " + str(idc))
                if(idc != (datacentre.index)):
                    #print(str(idc) + " " + str(datacentre.index))
                    transmission_time_i += ids.size * 1024 / (bandwidth[idc-1][datacentre.index-1])
                    print("Transmitting dataset " + str(ids.index) +  " to datacentre " + str(datacentre.index))
                    
            
            for ods in task.ods:
                dataset_to_datacenter[ods.index-1] = datacentre.index
                print("Adding output dataset " + str(ods.index) + " to datacentre " + str(datacentre.index))
                
            print("Appending " + str(transmission_time_i) + " " + str(datacentre.index))
            transmission_times[task.index-1] = (transmission_time_i)
            task_datacentres[task.index-1] = (datacentre.index)
            
        
        else:
            transmission_time_i = 10000
            datacentre = None
            
            for j in range(NUM_DATACENTRES):
                task_dc = datacentres[j]
                transmission_time_dcj = 0
                print("Placing task in datacentre " + str(task_dc.index))
          
                for ids in task.ids:
                    print("Input dataset " + str(ids.index))
                    idc = dataset_to_datacenter[ids.index-1]
                    print("Input dataset resides in " + str(idc))
                    if(idc != task_dc.index):
                        transmission_time_dcj += ids.size * 1024 / (bandwidth[idc-1][task_dc.index-1])
                        print("Transmitting dataset " + str(ids.index) +  " to datacentre " + str(task_dc.index) 
                             + " in " + str(ids.size * 1024 / (bandwidth[idc-1][task_dc.index-1])))
                   
                print("Trans time for task " + str(task.index) + " in datacentre " + str(j+1) + " is " 
                      + str(transmission_time_dcj))
                if (transmission_time_dcj < transmission_time_i):
                    print("Here " + str(task_dc.index))
                    transmission_time_i = transmission_time_dcj
                    datacentre = task_dc
                    
            for ods in task.ods:
                dataset_to_datacenter[ods.index-1] = datacentre.index-1
                print("Adding output dataset " + str(ods.index) + " to datacentre " + str(datacentre.index))
                
            print("Appending " + str(transmission_time_i) + " " + str(datacentre.index))
            transmission_times[task.index-1] = (transmission_time_i)
            task_datacentres[task.index-1] = (datacentre.index)
            
    print(transmission_times)
    print(task_datacentres)  
    
    return transmission_times, task_datacentres
            

transmission_times, task_datacentres = place_tasks([1,1,3,2,3])
total_transmission_time = np.sum(transmission_times)
print(total_transmission_time)

[1, 1, 3, 2, 3, 0]
[0, 0, 0, 0, 0]
Task - 5
Private ids 5 found in datacenter 3
Input dataset 5
Input dataset resides in 3
Adding output dataset 6 to datacentre 3
Appending 0 3
Task - 4
Private ids 4 found in datacenter 2
Input dataset 3
Input dataset resides in 3
Transmitting dataset 3 to datacentre 2
Input dataset 4
Input dataset resides in 2
Input dataset 6
Input dataset resides in 3
Transmitting dataset 6 to datacentre 2
Appending 75.09333333333333 2
Task - 3
Placing task in datacentre 1
Input dataset 1
Input dataset resides in 1
Input dataset 2
Input dataset resides in 1
Input dataset 3
Input dataset resides in 3
Transmitting dataset 3 to datacentre 1 in 153.6
Input dataset 6
Input dataset resides in 3
Transmitting dataset 6 to datacentre 1 in 409.6
Trans time for task 3 in datacentre 1 is 563.2
Here 1
Placing task in datacentre 2
Input dataset 1
Input dataset resides in 1
Transmitting dataset 1 to datacentre 2 in 307.2
Input dataset 2
Input dataset resides in 1
Transmitting datas

In [263]:
INITIAL_POPULATION = 10

MAX_W = 0.9
MIN_W = 0.4

C1_START = 0.9
C1_END = 0.4

C2_START = 0.9
C2_END = 0.4

In [271]:
particles = []
def initialize_population():
    for i in range(INITIAL_POPULATION):
        particle = np.random.randint(1, NUM_DATACENTRES+1, NUM_DATASETS)
        particle[3] = 2
        particle[4] = 3
        particles.append(particle)
        print(particle)
        
def particle_feasibility(particle):
    
    datacentre_usage = [0] * NUM_DATACENTRES
        
    for i in range(len(particle)):
        if((datacentre_usage[particle[i]-1] + datasets[i].size) > datacentres[particle[i]-1].capacity):
            return False
        datacentre_usage[particle[i]-1] += datasets[i].size 
        print("Data centre usage of data centre " + str(datacentres[particle[i]-1].index) + " is " + str(datacentre_usage[particle[i] - 1]))
        
    return True

def fitness(particle):
    
    if(particle_feasibility(particle) == False):
        return 1, 10000
    
    total_transmission_time, _ = place_tasks(particle)
    
    return 0, np.sum(total_transmission_time)
    

In [272]:
initialize_population()

for i in range(len(particles)):
    print("Fitness is " + str(fitness(particles[i])))

print("Fitness is " + str(fitness([1,1,1,1,1])))

[1 1 3 2 3]
[1 2 2 2 3]
[1 1 2 2 3]
[3 2 2 2 3]
[2 3 3 2 3]
[2 3 1 2 3]
[3 2 1 2 3]
[3 2 1 2 3]
[2 3 2 2 3]
[2 2 2 2 3]
Data centre usage of data centre 1 is 3
Data centre usage of data centre 1 is 8
Data centre usage of data centre 3 is 3
Data centre usage of data centre 2 is 3
Data centre usage of data centre 3 is 8
[1, 1, 3, 2, 3, 0]
[0, 0, 0, 0, 0]
Task - 5
Private ids 5 found in datacenter 3
Input dataset 5
Input dataset resides in 3
Adding output dataset 6 to datacentre 3
Appending 0 3
Task - 4
Private ids 4 found in datacenter 2
Input dataset 3
Input dataset resides in 3
Transmitting dataset 3 to datacentre 2
Input dataset 4
Input dataset resides in 2
Input dataset 6
Input dataset resides in 3
Transmitting dataset 6 to datacentre 2
Appending 75.09333333333333 2
Task - 3
Placing task in datacentre 1
Input dataset 1
Input dataset resides in 1
Input dataset 2
Input dataset resides in 1
Input dataset 3
Input dataset resides in 3
Transmitting dataset 3 to datacentre 1 in 153.6
Input 

In [ ]:
# Pep8 standards - https://www.python.org/dev/peps/pep-0008/#id39 
# Enum -https://docs.python.org/3/library/enum.html#creating-an-enum 
# Keyword arguments for Dataset (any class with large number of args)
# Pass arrays to function instead of global
# Timesheets